In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,28482
2,Huelva,Confirmados PDIA 14 días,1093
3,Huelva,Tasa PDIA 14 días,"212,98984741898397"
4,Huelva,Confirmados PDIA 7 días,447
5,Huelva,Total Confirmados,28657
6,Huelva,Curados,24264
7,Huelva,Fallecidos,364


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  28482.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7387.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1332 personas en los últimos 7 días 

Un positivo PCR cada 566 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,28482.0,447.0,1093.0,513170.0,87.105638,212.989847,71.0
Condado-Campiña,8956.0,227.0,573.0,156231.0,145.297668,366.764599,39.0
Huelva-Costa,16615.0,182.0,409.0,289548.0,62.856590,141.254645,30.0
Huelva (capital),7387.0,108.0,254.0,143837.0,75.084992,176.588778,18.0
Bollullos Par del Condado,809.0,34.0,71.0,14387.0,236.324460,493.501077,8.0
Almonte,1498.0,54.0,168.0,24507.0,220.345207,685.518423,5.0
Rociana del Condado,512.0,16.0,43.0,7939.0,201.536717,541.629928,5.0
Moguer,1096.0,34.0,77.0,21867.0,155.485435,352.128779,5.0
Palma del Condado (La),1035.0,7.0,17.0,10801.0,64.808814,157.392834,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,231.0,25.0,42.0,3261.0,766.636001,1287.948482,1.0
Cumbres Mayores,49.0,1.0,22.0,1749.0,57.175529,1257.861635,0.0
Cortelazor,7.0,2.0,3.0,299.0,668.896321,1003.344482,0.0
Galaroza,27.0,2.0,10.0,1382.0,144.717800,723.589001,0.0
Almonte,1498.0,54.0,168.0,24507.0,220.345207,685.518423,5.0
Rociana del Condado,512.0,16.0,43.0,7939.0,201.536717,541.629928,5.0
Beas,197.0,11.0,23.0,4341.0,253.397835,529.831836,0.0
Cañaveral de León,5.0,2.0,2.0,396.0,505.050505,505.050505,1.0
Bollullos Par del Condado,809.0,34.0,71.0,14387.0,236.324460,493.501077,8.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,49.0,1.0,22.0,1749.0,57.175529,1257.861635,0.0,0.045455
Niebla,193.0,1.0,6.0,4116.0,24.295432,145.772595,0.0,0.166667
Gibraleón,585.0,4.0,21.0,12737.0,31.404569,164.873989,3.0,0.190476
Galaroza,27.0,2.0,10.0,1382.0,144.717800,723.589001,0.0,0.200000
Aljaraque,993.0,3.0,14.0,21474.0,13.970383,65.195120,1.0,0.214286
Aracena,508.0,2.0,9.0,8255.0,24.227741,109.024833,1.0,0.222222
Almonte,1498.0,54.0,168.0,24507.0,220.345207,685.518423,5.0,0.321429
Aroche,90.0,1.0,3.0,3054.0,32.743942,98.231827,0.0,0.333333
Palos de la Frontera,580.0,7.0,21.0,11742.0,59.615057,178.845171,3.0,0.333333
